In [ ]:
import os
import speech_recognition as sr
from pydub import AudioSegment

In [ ]:
def transcribe_audio(file_path):
    """Transcribe audio to text."""
    recognizer = sr.Recognizer()

    # Load audio file
    audio = None
    if file_path.endswith(".mp3"):
        # If the audio file is mp3, convert it to WAV first
        audio = AudioSegment.from_mp3(file_path)
        audio_path = file_path.replace(".mp3", ".wav")
        audio.export(audio_path, format="wav")
        file_path = audio_path  # Update the file_path to the new wav file

    # Load the audio file into the recognizer
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)

    try:
        # Recognize speech using Google Web Speech API
        print(f"Transcribing {file_path}...")
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Audio Unrecognizable"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

def transcribe_audio_files_from_directory(directory_path):
    """Read all audio files in the directory and return their transcripts."""
    transcripts = {}
    
    # Walk through the directory and find audio files
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Check if it's an audio file (can add more formats if needed)
        if filename.endswith(('.wav', '.mp3')):
            transcript = transcribe_audio(file_path)
            transcripts[filename] = transcript

    return transcripts

if __name__ == "__main__":
    # Path to your audio files
    audio_directory = './data/audio'
    
    if os.path.exists(audio_directory):
        # Transcribe all audio files in the directory
        transcripts = transcribe_audio_files_from_directory(audio_directory)
        
        # Print or save the transcripts
        for filename, transcript in transcripts.items():
            print(f"Transcript for {filename}:\n{transcript}\n")
    else:
        print(f"The directory {audio_directory} does not exist.")
